<a href="https://colab.research.google.com/github/Ahmad-abdulrahim/Personalized-Job-Matching-System/blob/Development/PJMS_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical computations
import re           # For regular expressions
import nltk         # Natural Language Toolkit
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.corpus import stopwords   # Stopwords
from nltk.tokenize import word_tokenize  # Tokenization
from nltk.stem import WordNetLemmatizer  # Lemmatization
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
df = pd.read_csv("/content/Job Postings Dataset.csv")
df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [ ]:
df.columns

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')

In [ ]:
df.shape

(19001, 24)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19001 entries, 0 to 19000
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   jobpost           19001 non-null  object
 1   date              19001 non-null  object
 2   Title             18973 non-null  object
 3   Company           18994 non-null  object
 4   AnnouncementCode  1208 non-null   object
 5   Term              7676 non-null   object
 6   Eligibility       4930 non-null   object
 7   Audience          640 non-null    object
 8   StartDate         9675 non-null   object
 9   Duration          10798 non-null  object
 10  Location          18969 non-null  object
 11  JobDescription    15109 non-null  object
 12  JobRequirment     16479 non-null  object
 13  RequiredQual      18517 non-null  object
 14  Salary            9622 non-null   object
 15  ApplicationP      18941 non-null  object
 16  OpeningDate       18295 non-null  object
 17  Deadline    

In [ ]:
# Dropping unnecessary columns from dataframe
df_clean = df.drop(columns=['date', 'AnnouncementCode',
                            'StartDate', 'ApplicationP',
                            'OpeningDate', 'Deadline',
                            'Notes', 'Attach', 'Year',
                            'Month', 'AboutC', 'jobpost',
                            'Term', 'Eligibility', 'Audience',
                            'Duration', 'Location', 'Salary'])

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19001 entries, 0 to 19000
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           18973 non-null  object
 1   Company         18994 non-null  object
 2   JobDescription  15109 non-null  object
 3   JobRequirment   16479 non-null  object
 4   RequiredQual    18517 non-null  object
 5   IT              19001 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 760.9+ KB


In [ ]:
df_clean.columns = df_clean.columns.str.strip()
df_clean.rename(columns={'RequiredQual': 'RequiredQualifications', 'JobRequirment': 'JobRequirement'}, inplace=True)

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19001 entries, 0 to 19000
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Title                   18973 non-null  object
 1   Company                 18994 non-null  object
 2   JobDescription          15109 non-null  object
 3   JobRequirement          16479 non-null  object
 4   RequiredQualifications  18517 non-null  object
 5   IT                      19001 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 760.9+ KB


In [ ]:
# List of columns to consider for dropping rows
columns_to_check = ['Title', 'Company', 'JobDescription', 'JobRequirement', 'RequiredQualifications']

# Drop rows with null values in the specified columns
df_clean.dropna(subset=columns_to_check, inplace=True)

# Reset index after dropping rows
df_clean.reset_index(drop=True, inplace=True)

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13124 entries, 0 to 13123
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Title                   13124 non-null  object
 1   Company                 13124 non-null  object
 2   JobDescription          13124 non-null  object
 3   JobRequirement          13124 non-null  object
 4   RequiredQualifications  13124 non-null  object
 5   IT                      13124 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 525.6+ KB


In [ ]:
df_clean.head()

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])  # Disable parser and NER for speed

In [ ]:
def preprocess_text_spacy(text):
    """
    Advanced text preprocessing using spaCy.
    """
    # Initial cleaning (remove punctuation and numbers, optional)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    # Process text with spaCy
    doc = nlp(text)

    # Lemmatize and remove stopwords and punctuations
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Join processed tokens back into string
    return " ".join(tokens)

def preprocess_dataframe(df, text_columns):
    """
    Apply text preprocessing to specified text columns of a pandas DataFrame.
    """
    # Use .applymap() for elementwise operation if necessary
    for col in text_columns:
        df[col] = df[col].apply(preprocess_text_spacy)
    return df

In [ ]:
text_columns = ['JobRequirement', 'JobDescription', 'RequiredQualifications', 'Title', 'Company']
df_preprocessed = preprocess_dataframe(df_clean.copy(), text_columns)

In [ ]:
df_preprocessed.head()

,Title,Company,JobDescription,JobRequirement,RequiredQualifications,IT
0,chief financial officer,ameria investment consult company,ameria investment consult company seek chief f...,supervise financial management administrative ...,perform job successfully individual able perfo...,False
1,country coordinator,caucasus environmental ngo network cenn,public outreach strengthen grow network enviro...,work country director provide environmental in...,degree environmentally related field year rele...,False
2,bcc specialist,manoff group,lead local enhancement development health bcc ...,identify gap knowledge oversee information col...,advanced degree public health social science c...,False
3,community development capacity building conflict,food security regional cooperation stability p...,food security regional cooperation stability p...,assist tavush marz community community union d...,high education andor professional experience e...,False
4,country economist nob,united nations development programme armenia,united nations development programme armenia a...,incumbent direct supervision undp resident rep...,minimum master degree economic minimum year re...,False


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Concatenate text columns
df_preprocessed['combined_text'] = df_preprocessed['JobDescription'] + ' ' + df_preprocessed['JobRequirement'] + ' ' + df_preprocessed['RequiredQualifications']

# Apply TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(df_preprocessed['combined_text'])

In [ ]:
"""
def clean_text(text):
    """
   # Perform basic text cleaning operations
    """
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

def tokenize_text(text):
    """
  #  Tokenize the text into individual words or tokens
    """
    tokens = word_tokenize(text)
    return tokens

def remove_stopwords(tokens):
    """
   # Remove stopwords from list of tokens
    """
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

def lemmatize_tokens(tokens):
    """
   # Lemmatize tokens to reduce them to their base or root form
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens

def preprocess_text(text):
    """
   # Perform full preprocessing pipeline on the input text.
    """
    # Clean the text
    cleaned_text = clean_text(text)
    # Tokenize the cleaned text
    tokens = tokenize_text(cleaned_text)
    # Remove stopwords from the tokens
    filtered_tokens = remove_stopwords(tokens)
    # Lemmatize the filtered tokens
    lemmatized_tokens = lemmatize_tokens(filtered_tokens)
    # Join the lemmatized tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

def preprocess_text_df(df):
    """
    #Perform full preprocessing pipeline on text columns on DataFrame
    """
    # Copy the input DataFrame to avoid modifying the original data
    df_processed = df.copy()

    # Filter columns with text data
    text_columns = [col for col in df_processed.columns if df_processed[col].dtype == 'object']

    # Iterate over text columns and preprocess text data
    for col in text_columns:
        df_processed[col] = df_processed[col].apply(lambda x: preprocess_text(x) if isinstance(x, str) else x)

    return df_processed
"""